In [9]:
from modules import download_gaia_vari_short_timescale, merge_gaia_vari_short_timescale
download_gaia_vari_short_timescale()
merged_data_path = merge_gaia_vari_short_timescale()

Download complete. Saved at ./data/vari_short_timescale_files
File vari_short_timescale_files.parquet already exist


In [10]:
import pandas as pd

merged_data = pd.read_parquet(merged_data_path)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471679 entries, 0 to 471678
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   solution_id                471679 non-null  int64  
 1   source_id                  471679 non-null  int64  
 2   amplitude_estimate         471679 non-null  float64
 3   number_of_fov_transits     471679 non-null  int64  
 4   mean_of_fov_abbe_values    471679 non-null  float64
 5   variogram_num_points       471679 non-null  int64  
 6   variogram_char_timescales  471679 non-null  object 
 7   variogram_values           471679 non-null  object 
 8   frequency                  471596 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 32.4+ MB


In [11]:
merged_data.dropna()
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 471679 entries, 0 to 471678
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   solution_id                471679 non-null  int64  
 1   source_id                  471679 non-null  int64  
 2   amplitude_estimate         471679 non-null  float64
 3   number_of_fov_transits     471679 non-null  int64  
 4   mean_of_fov_abbe_values    471679 non-null  float64
 5   variogram_num_points       471679 non-null  int64  
 6   variogram_char_timescales  471679 non-null  object 
 7   variogram_values           471679 non-null  object 
 8   frequency                  471596 non-null  float64
dtypes: float64(3), int64(4), object(2)
memory usage: 32.4+ MB


In [13]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers


# Handle empty variogram lists
merged_data['variogram_char_timescales'] = merged_data['variogram_char_timescales'].apply(
    lambda x: eval(x) if isinstance(x, str) and len(eval(x)) > 0 else [0]
)
merged_data['variogram_values'] = merged_data['variogram_values'].apply(
    lambda x: eval(x) if isinstance(x, str) and len(eval(x)) > 0 else [0]
)

# Calculate features
merged_data['variogram_char_timescales_mean'] = merged_data['variogram_char_timescales'].apply(np.mean)
merged_data['variogram_values_mean'] = merged_data['variogram_values'].apply(lambda x: np.mean([float(i) for i in x]))

# Drop rows only where critical columns are NaN
final_data_cleaned = merged_data.dropna(subset=['frequency'])

# Encode labels
label_encoder = LabelEncoder()
X = final_data_cleaned[['amplitude_estimate', 'frequency', 'variogram_char_timescales_mean', 'variogram_values_mean']]
y = final_data_cleaned['number_of_fov_transits']
y_encoded = label_encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Build and train the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=10, validation_split=0.2)

# Evaluate
test_acc = model.evaluate(X_test_scaled, y_test)[1]
print(f"Test Accuracy: {test_acc:.4f}")

Epoch 1/10


/home/andredon/projects/University/Practice4/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9432/9432 ━━━━━━━━━━━━━━━━━━━━ 4s 372us/step - accuracy: 0.0339 - loss: 4.0447 - val_accuracy: 0.0342 - val_loss: 3.9681
Epoch 2/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 363us/step - accuracy: 0.0354 - loss: 3.9629 - val_accuracy: 0.0354 - val_loss: 3.9588
Epoch 3/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 365us/step - accuracy: 0.0355 - loss: 3.9620 - val_accuracy: 0.0350 - val_loss: 3.9545
Epoch 4/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 362us/step - accuracy: 0.0364 - loss: 3.9511 - val_accuracy: 0.0353 - val_loss: 3.9482
Epoch 5/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 369us/step - accuracy: 0.0363 - loss: 3.9512 - val_accuracy: 0.0361 - val_loss: 3.9473
Epoch 6/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 362us/step - accuracy: 0.0369 - loss: 3.9458 - val_accuracy: 0.0348 - val_loss: 3.9458
Epoch 7/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 368us/step - accuracy: 0.0363 - loss: 3.9436 - val_accuracy: 0.0370 - val_loss: 3.9397
Epoch 8/10
9432/9432 ━━━━━━━━━━━━━━━━━━━━ 3s 370us/step - accuracy: 0.0369 - loss: 3.93